In [1]:
import numpy as np

!pip install folium
import folium
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors
import types
import pandas as pd
from botocore.client import Config
import ibm_boto3
import requests


     |████████████████████████████████| 92kB 15.2MB/s eta 0:00:01


In [54]:
def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_7c79a88bb82342349f41a6c5fbc38d34 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='9k-vmctrDlw8Y3kVx86aWcoLE8BE4fm_vvs68PxpPoQi',
    ibm_auth_endpoint="https://iam.ng.bluemix.net/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

body = client_7c79a88bb82342349f41a6c5fbc38d34.get_object(Bucket='capstone-donotdelete-pr-judrj1vm8ftfvc',Key='Pittsburgh.xlsx')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df = pd.read_excel(body)
df.head()


,Neighborhood,Lat,Long
0,Allegheny Center,40.453,-80.005
1,Allegheny West,40.452,-80.016
2,Allentown,40.421,-79.994
3,Arlington,40.415,-79.970
4,Banksville,40.412,-80.039


In [56]:
df.shape

(78, 3)

In [57]:
latitude = 40.4406
longitude = -79.9959

map_pittsburgh = folium.Map(location=[latitude, longitude], zoom_start=12)

for lat, lng, label in zip(df['Lat'], df['Long'], df['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_pittsburgh)  
    
map_pittsburgh


In [5]:
CLIENT_ID = 'CR04UIFO54W2VCBAXTCP1KVEBF0WGTGMZ0LVQ3OQ44BH3VK3'
CLIENT_SECRET = 'TRV15GMB35MZQJMMWQL0YLXNMWMZWPEPELDSVULO2KUDCL2I'
VERSION = '20200419'

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: CR04UIFO54W2VCBAXTCP1KVEBF0WGTGMZ0LVQ3OQ44BH3VK3
CLIENT_SECRET:TRV15GMB35MZQJMMWQL0YLXNMWMZWPEPELDSVULO2KUDCL2I


In [6]:
def getNearbyVenues(names, latitudes, longitudes,radius=500,LIMIT=100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
         
            
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [7]:
venues = getNearbyVenues(names=df['Neighborhood'],
                                   latitudes=df['Lat'],
                                   longitudes=df['Long'])

Allegheny Center
Allegheny West
Allentown
Arlington
Banksville
Beechview
Beltzhoover
Bloomfield
Bluff
Bon Air
Brighton Heights
Brookline
California-Kirkbride
Carrick
Central Business District
Central Lawrenceville
Central Northside
Chartiers City
Chateau
Crafton Heights
Duquesne Heights
East Allegheny
East Carnegie
East Hills
East Liberty
Elliott
Esplen
Fairywood
Fineview
Friendship
Garfield
Glen Hazel
Greenfield
Hays
Hazelwood
Highland Park
Hill District
Homewood
Knoxville
Larimer
Lincoln Place
Lincoln-Lemington-Belmar
Lawrenceville
Manchester
Marshall-Shadeland
Morningside
Mount Washington
Mt. Oliver
New Homestead
North Shore
Northview Heights
Oakland
Oakwood
Overbrook
Perry North
Perry South
Point Breeze
Point Breeze North
Polish Hill
Regent Square
Ridgemont
Shadyside
Sheraden
South Shore
South Side Flats
South Side Slopes
Spring Garden
Spring Hill-City View
Squirrel Hill
St. Clair
Stanton Heights
Strip District
Summer Hill
Swisshelm Park
Troy Hill
West End
Westwood
Windgap


In [8]:
venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Allegheny Center,40.453,-80.005,Children's Museum of Pittsburgh,40.452793,-80.006569,Museum
1,Allegheny Center,40.453,-80.005,Federal Galley,40.451605,-80.006045,Food Court
2,Allegheny Center,40.453,-80.005,Park House,40.453284,-80.001504,Bar
3,Allegheny Center,40.453,-80.005,El Burro,40.455860,-80.006689,Mexican Restaurant
4,Allegheny Center,40.453,-80.005,Bistro To Go,40.453450,-80.000995,Deli / Bodega


In [9]:
onehot = pd.get_dummies(venues['Venue Category'])

onehot['Neighborhood'] = venues['Neighborhood'] 

hood = onehot['Neighborhood']
onehot.drop(labels=['Neighborhood'], axis=1,inplace = True)
onehot.insert(0, 'Neighborhood', hood)

onehot.head()

,Neighborhood,Adult Boutique,American Restaurant,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,...,Turkish Restaurant,Used Bookstore,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wings Joint,Women's Store,Yoga Studio,Zoo
0,Allegheny Center,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Allegheny Center,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Allegheny Center,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Allegheny Center,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Allegheny Center,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [10]:
pittsburgh_grouped = onehot.groupby('Neighborhood').mean().reset_index()
pittsburgh_grouped

,Neighborhood,Adult Boutique,American Restaurant,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,...,Turkish Restaurant,Used Bookstore,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wings Joint,Women's Store,Yoga Studio,Zoo
0,Allegheny Center,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.035714,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.035714
1,Allegheny West,0.00,0.086957,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,Allentown,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.166667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,Arlington,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,Banksville,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,Beechview,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,Beltzhoover,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,Bloomfield,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.027027,0.000000,0.000000,0.000000,0.000000,0.000000
8,Bluff,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,Bon Air,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [11]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [12]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = pittsburgh_grouped['Neighborhood']

for ind in np.arange(pittsburgh_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(pittsburgh_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Allegheny Center,Deli / Bodega,Park,Exhibit,Zoo,Bar,Café,Burger Joint,Brewery,Pizza Place,Food Court
1,Allegheny West,American Restaurant,Pub,Sandwich Place,Fast Food Restaurant,Food Truck,Thai Restaurant,Diner,Deli / Bodega,Pharmacy,Restaurant
2,Allentown,Discount Store,Chinese Restaurant,Vegetarian / Vegan Restaurant,Italian Restaurant,Coffee Shop,Caribbean Restaurant,Football Stadium,Food Truck,Food Service,Food Court
3,Arlington,Farm,Cosmetics Shop,Farmers Market,Fountain,Football Stadium,Food Truck,Food Service,Food Court,Food & Drink Shop,Food
4,Banksville,Park,Pool,Pizza Place,Zoo,Food Truck,Food Service,Food Court,Food & Drink Shop,Food,Flower Shop


In [13]:
kclusters = 8

pittsburgh_grouped_clustering = pittsburgh_grouped.drop('Neighborhood', 1)

kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(pittsburgh_grouped_clustering)

kmeans.labels_[0:10] 

array([2, 2, 2, 6, 2, 0, 2, 2, 0, 2], dtype=int32)

In [14]:
neighborhoods_venues_sorted.insert(0, 'Cluster_Labels', kmeans.labels_)

pittsburgh_merged = df

#merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
pittsburgh_merged = pittsburgh_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

pittsburgh_merged.head() # check the last columns!

,Neighborhood,Lat,Long,Cluster_Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Allegheny Center,40.453,-80.005,2.0,Deli / Bodega,Park,Exhibit,Zoo,Bar,Café,Burger Joint,Brewery,Pizza Place,Food Court
1,Allegheny West,40.452,-80.016,2.0,American Restaurant,Pub,Sandwich Place,Fast Food Restaurant,Food Truck,Thai Restaurant,Diner,Deli / Bodega,Pharmacy,Restaurant
2,Allentown,40.421,-79.994,2.0,Discount Store,Chinese Restaurant,Vegetarian / Vegan Restaurant,Italian Restaurant,Coffee Shop,Caribbean Restaurant,Football Stadium,Food Truck,Food Service,Food Court
3,Arlington,40.415,-79.970,6.0,Farm,Cosmetics Shop,Farmers Market,Fountain,Football Stadium,Food Truck,Food Service,Food Court,Food & Drink Shop,Food
4,Banksville,40.412,-80.039,2.0,Park,Pool,Pizza Place,Zoo,Food Truck,Food Service,Food Court,Food & Drink Shop,Food,Flower Shop


In [20]:
pittsburgh_merged['Cluster_Labels'] = pittsburgh_merged['Cluster_Labels'].astype(str)

In [34]:
Cluster_1 = pittsburgh_merged.loc[pittsburgh_merged['Cluster_Labels'] == '0.0'].copy()
Cluster_1

,Neighborhood,Lat,Long,Cluster_Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Beechview,40.414,-80.022,0.0,Playground,Supermarket,Taco Place,Light Rail Station,Park,Bar,Zoo,Farmers Market,Food Truck,Food Service
8,Bluff,40.436,-79.989,0.0,Pizza Place,Bar,Hotel,Sporting Goods Shop,Food Service,Snack Place,Bus Station,Lounge,Outdoor Sculpture,Hockey Arena
18,Chateau,40.449,-80.028,0.0,Harbor / Marina,Scenic Lookout,Café,Fabric Shop,Food Truck,Food Service,Food Court,Food & Drink Shop,Food,Flower Shop
19,Crafton Heights,40.445,-80.052,0.0,American Restaurant,Bar,Zoo,Farmers Market,Fountain,Football Stadium,Food Truck,Food Service,Food Court,Food & Drink Shop
32,Greenfield,40.422,-79.942,0.0,Bar,Baseball Field,Pizza Place,College Cafeteria,Gift Shop,Theater,Zoo,Football Stadium,Food Truck,Food Service
37,Homewood,40.460,-79.902,0.0,Sports Bar,BBQ Joint,Farm,Fountain,Football Stadium,Food Truck,Food Service,Food Court,Food & Drink Shop,Food
44,Marshall-Shadeland,40.469,-80.032,0.0,Bar,Café,Baseball Field,Grocery Store,Zoo,Fountain,Football Stadium,Food Truck,Food Service,Food Court
45,Morningside,40.484,-79.928,0.0,Baseball Field,Rental Car Location,Bar,Tanning Salon,Salon / Barbershop,Pharmacy,Convenience Store,Flower Shop,Farm,Food
46,Mount Washington,40.428,-80.011,0.0,Bar,Diner,Baseball Field,Bakery,Coffee Shop,Russian Restaurant,Sandwich Place,Gastropub,Beer Garden,Sports Bar
47,Mt. Oliver,40.412,-79.980,0.0,Moving Target,Construction & Landscaping,Bar,Farmers Market,Fountain,Football Stadium,Food Truck,Food Service,Food Court,Food & Drink Shop


In [35]:
Cluster_2 = pittsburgh_merged.loc[pittsburgh_merged['Cluster_Labels'] == '1.0'].copy()
Cluster_2

,Neighborhood,Lat,Long,Cluster_Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
43,Manchester,40.455,-80.025,1.0,Storage Facility,Baseball Field,Zoo,Farmers Market,Football Stadium,Food Truck,Food Service,Food Court,Food & Drink Shop,Food
77,Windgap,40.455,-80.074,1.0,Baseball Field,Zoo,Fried Chicken Joint,Fountain,Football Stadium,Food Truck,Food Service,Food Court,Food & Drink Shop,Food


In [36]:
Cluster_3 = pittsburgh_merged.loc[pittsburgh_merged['Cluster_Labels'] == '2.0'].copy()
Cluster_3

,Neighborhood,Lat,Long,Cluster_Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Allegheny Center,40.453,-80.005,2.0,Deli / Bodega,Park,Exhibit,Zoo,Bar,Café,Burger Joint,Brewery,Pizza Place,Food Court
1,Allegheny West,40.452,-80.016,2.0,American Restaurant,Pub,Sandwich Place,Fast Food Restaurant,Food Truck,Thai Restaurant,Diner,Deli / Bodega,Pharmacy,Restaurant
2,Allentown,40.421,-79.994,2.0,Discount Store,Chinese Restaurant,Vegetarian / Vegan Restaurant,Italian Restaurant,Coffee Shop,Caribbean Restaurant,Football Stadium,Food Truck,Food Service,Food Court
4,Banksville,40.412,-80.039,2.0,Park,Pool,Pizza Place,Zoo,Food Truck,Food Service,Food Court,Food & Drink Shop,Food,Flower Shop
6,Beltzhoover,40.416,-80.003,2.0,Park,Cosmetics Shop,Health & Beauty Service,Tennis Court,Zoo,Food Truck,Food Service,Food Court,Food & Drink Shop,Food
7,Bloomfield,40.461,-79.948,2.0,Bar,Bakery,Thai Restaurant,Pizza Place,Bookstore,Sandwich Place,Coffee Shop,Grocery Store,New American Restaurant,Italian Restaurant
9,Bon Air,40.408,-80.000,2.0,Park,Baseball Field,Home Service,Playground,Light Rail Station,Tennis Court,Food Truck,Food Service,Food Court,Food & Drink Shop
10,Brighton Heights,40.484,-80.037,2.0,Miscellaneous Shop,Bus Station,Market,Pharmacy,Park,Seafood Restaurant,Zoo,Farmers Market,Food Service,Food Court
12,California-Kirkbride,40.461,-80.023,2.0,Bakery,Home Service,Farm,Fountain,Football Stadium,Food Truck,Food Service,Food Court,Food & Drink Shop,Food
13,Carrick,40.397,-79.987,2.0,Pizza Place,Bank,Business Service,Auto Garage,Baseball Field,Farmers Market,Football Stadium,Food Truck,Food Service,Food Court


In [37]:
Cluster_4 = pittsburgh_merged.loc[pittsburgh_merged['Cluster_Labels'] == '3.0'].copy()
Cluster_4


,Neighborhood,Lat,Long,Cluster_Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
39,Larimer,40.464,-79.913,3.0,Coffee Shop,Zoo,Farmers Market,Fountain,Football Stadium,Food Truck,Food Service,Food Court,Food & Drink Shop,Food


In [38]:
Cluster_5 = pittsburgh_merged.loc[pittsburgh_merged['Cluster_Labels'] == '4.0'].copy()
Cluster_5

,Neighborhood,Lat,Long,Cluster_Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
55,Perry South,40.468,-80.015,4.0,Scenic Lookout,Pool,Bed & Breakfast,Zoo,Farmers Market,Football Stadium,Food Truck,Food Service,Food Court,Food & Drink Shop
70,Stanton Heights,40.480,-79.938,4.0,Restaurant,Scenic Lookout,Zoo,Fabric Shop,Food Truck,Food Service,Food Court,Food & Drink Shop,Food,Flower Shop


In [39]:
Cluster_6 = pittsburgh_merged.loc[pittsburgh_merged['Cluster_Labels'] == '5.0'].copy()
Cluster_6

,Neighborhood,Lat,Long,Cluster_Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
41,Lincoln-Lemington-Belmar,40.469,-79.896,5.0,Park,Zoo,Hotel Bar,Football Stadium,Food Truck,Food Service,Food Court,Food & Drink Shop,Food,Flower Shop


In [40]:
Cluster_7 = pittsburgh_merged.loc[pittsburgh_merged['Cluster_Labels'] == '6.0'].copy()
Cluster_7

,Neighborhood,Lat,Long,Cluster_Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Arlington,40.415,-79.97,6.0,Farm,Cosmetics Shop,Farmers Market,Fountain,Football Stadium,Food Truck,Food Service,Food Court,Food & Drink Shop,Food


In [41]:
Cluster_8 = pittsburgh_merged.loc[pittsburgh_merged['Cluster_Labels'] == '7.0'].copy()
Cluster_8

,Neighborhood,Lat,Long,Cluster_Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
22,East Carnegie,40.42,-80.071,7.0,Pub,Liquor Store,Zoo,Farmers Market,Football Stadium,Food Truck,Food Service,Food Court,Food & Drink Shop,Food


In [49]:
latitude_pitt = 40.4444
longitude_pitt = -79.9608

map_neighborhoods = folium.Map(location=[latitude_pitt, longitude_pitt], zoom_start=14)

for lat, lng, label in zip(Cluster_3['Lat'], Cluster_3['Long'], Cluster_3['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_neighborhoods)  
    
map_neighborhoods